In [ ]:
from google.colab import drive
drive.mount('/content/drive')


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import itertools

import time
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import gym
import random
import sklearn.preprocessing
import sklearn.pipeline
import torch
import random
import warnings
import copy
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import math



warnings.filterwarnings("ignore", category=DeprecationWarning)
standard = StandardScaler()
minmax = MinMaxScaler()

 
import json
from IOHMM import UnSupervisedIOHMM
from IOHMM import OLS, CrossEntropyMNL
from hmmlearn import _hmmc



In [ ]:
# src = '/content/drive/MyDrive/Seminar/seminar-10'
src = '/content/drive/MyDrive/seminar-10'


In [ ]:
dataset = f"{src}/train_FD001.txt"


df_full = pd.read_csv( dataset, sep=" ", header=None, skipinitialspace=True).dropna(axis=1)
df_full = df_full.rename(columns={0: 'unit', 1: 'cycle', 2: 'W1', 3: 'W2', 4: 'W3'})
# Filter df_full for engine units 1 to 5
df_full = df_full[df_full['unit'].isin(range(1,16))].reset_index(drop=True)

df = df_full
df_full.columns = df_full.columns.astype(str)

df_A = df[df.columns[[0, 1]]] # unit and cycle
df_W = df[df.columns[[2, 3, 4]]] # W1 w2 w3 (opration conditions)
df_S = df[df.columns[list(range(5, 26))]] #sensors
df_X = pd.concat([df_W, df_S], axis=1)

df[['W1', 'W2', 'W3', *map(str, range(5, 26))]] = minmax.fit_transform(pd.concat([df_W, df_S], axis=1))

In [ ]:

cols_to_drop = df.nunique()[df.nunique() == 1].index
df = df.drop(cols_to_drop, axis=1)

cols_to_drop = df.nunique()[df.nunique() == 2].index
df = df.drop(cols_to_drop, axis=1)

In [ ]:

outputs = [col for col in df.columns if col.isdigit()]

df_obs = PCA(n_components=len(outputs)).fit_transform(df[np.array(outputs).squeeze()])
df_obs = pd.DataFrame(df_obs)
df_input = PCA(n_components=1).fit_transform(df[['W1', 'W2']])
df_hmm = df_obs
df_hmm['W'] = df_input


In [ ]:
outputs = [[m] for m in df_obs.head() if type(m) == int]


def split_X_lengths(X, lengths):
    if lengths is None:
        return [X]
    else:
        cs = np.cumsum(lengths)
        n_samples = len(X)
        if cs[-1] > n_samples:
            raise ValueError("more than {} samples in lengths array {}"
                             .format(n_samples, lengths))
        elif cs[-1] != n_samples:
            warnings.warn(
                "less that {} samples in lengths array {}; support for "
                "silently dropping samples is deprecated and will be removed"
                    .format(n_samples, lengths),
                DeprecationWarning, stacklevel=3)
        return np.split(X, cs)[:-1]

# # Initialize IOHMM with two states
np.Infinity = np.inf

#
num_states = 10
SHMM = UnSupervisedIOHMM(num_states=num_states ,max_EM_iter=30, EM_tol=1e-3 )
SHMM.set_models(model_emissions=[OLS(est_stderr=True), OLS(est_stderr=True), OLS(est_stderr=True), OLS(est_stderr=True),
                                 OLS(est_stderr=True), OLS(est_stderr=True), OLS(est_stderr=True), OLS(est_stderr=True),
                                 OLS(est_stderr=True), OLS(est_stderr=True), OLS(est_stderr=True), OLS(est_stderr=True),
                                 OLS(est_stderr=True), OLS(est_stderr=True)],
                model_transition=CrossEntropyMNL(solver='lbfgs'),
                model_initial=CrossEntropyMNL(solver='lbfgs'))

# We set operating conditions as the input covariate associate with the sensor output
SHMM.set_inputs(covariates_initial=[], covariates_transition=['W'],
                covariates_emissions=[['W']] * len(outputs))

#we use sensor data as outputs
SHMM.set_outputs(outputs)

# lengths = [192, 174, 256, 203, ...] per engine
lengths = [df_A[df_A['unit'] == i].cycle.max() for i in range(1, df_A['unit'].max() + 1)]
split_data = split_X_lengths(df_hmm, lengths)
SHMM.set_data(split_data)

SHMM.train()

In [ ]:

#  viterbi
log_gammas = SHMM.log_gammas  # A list of arrays, one for each sequence

# Convert log probabilities to normal probabilities and find the most likely states
state_sequences = [np.argmax(np.exp(sequence), axis=1) for sequence in log_gammas]
 
for i, seq in enumerate(state_sequences):
    print(f"Sequence {i+1}: {seq}")


state_sequences = []
log_gammas = SHMM.log_gammas
for i in range(len(SHMM.log_gammas)): # i => for loop on each engine unit
    for j in range(lengths[i]): # j => for loop on each timestep
        state_sequences.append(np.argmax(np.exp(log_gammas[i])[j]))

pred = [state_sequences[df[df['unit'] == i].index[0]:df[df['unit'] == i].index[-1] + 1] for i in
        range(1, len(SHMM.log_gammas) + 1)]


failure_states = [pred[i][-1] for i in range(len(SHMM.log_gammas) )]
failure_states

In [ ]:
hierarchical = []
h = []
failure = np.unique(failure_states)
failure



for i in range(len(pred)):
    h = []
    for j in range(len(failure)):
        h.append(np.where(np.array(pred[i]) == failure[j]))
    h = np.squeeze(np.sort(np.concatenate(h, 1)))
    hierarchical.append(h)



for i, seq in enumerate(hierarchical):
    print(f"Sequence {i+1}: {seq}")
    print(f"---------------------")

<hr/>
RL
<hr/>
